In [1]:
pip install opencv-python-headless numpy scikit-image


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import os
import pickle
from sklearn.cluster import KMeans

In [3]:
def extract_color_histogram(frame, bins=64):
    """Trích xuất histogram màu sắc từ khung hình trong không gian màu HSV và LAB.
       Sử dụng 64 bins cho mỗi kênh màu để thu thập thông tin chi tiết hơn."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    hsv_hist = cv2.calcHist([hsv], [0, 1, 2], None, [bins]*3, [0, 180, 0, 256, 0, 256])
    lab_hist = cv2.calcHist([lab], [0, 1, 2], None, [bins]*3, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hsv_hist, hsv_hist)
    cv2.normalize(lab_hist, lab_hist)
    combined_hist = np.hstack((hsv_hist.flatten(), lab_hist.flatten()))
    return combined_hist


In [4]:
def extract_brightness_contrast_features(frame):
    """Trích xuất độ sáng và độ tương phản trung bình từ khung hình."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    contrast = np.std(gray)
    return brightness, contrast

In [5]:
def extract_lbp_features(frame, radii=[1, 3, 8], n_points=[8, 16, 24], method='uniform'):
    """ Trích xuất đặc trưng Local Binary Patterns từ khung hình với nhiều bán kính và số điểm khác nhau. """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lbp_features = []
    for radius, points in zip(radii, n_points):
        lbp = local_binary_pattern(gray, points, radius, method)
        hist, _ = np.histogram(lbp.ravel(), bins=points+2, range=(0, points+2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)  # Chuẩn hóa histogram
        lbp_features.extend(hist)
    return np.array(lbp_features)


In [6]:
def extract_sift_features(frame, n_clusters=10):
    """Trích xuất và gom cụm đặc trưng SIFT từ khung hình, sử dụng K-means để giảm số lượng điểm đặc trưng."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is not None:
        if len(descriptors) > n_clusters:
            # Cập nhật n_init ở đây
            kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=0)
            kmeans.fit(descriptors)
            descriptors = kmeans.cluster_centers_
        else:
            # Trường hợp số lượng descriptors ít hơn số cụm yêu cầu, sử dụng chính các descriptors đó
            descriptors = np.mean(descriptors, axis=0, keepdims=True)
    else:
        # Nếu không có descriptors nào được tìm thấy, trả về None
        descriptors = None
    return descriptors


In [7]:
def process_video(video_path, sample_rate=30, bins=32, radii=[1, 3, 5], n_points=[8, 16, 24], sift_clusters=10):
    cap = cv2.VideoCapture(video_path)
    color_features = []
    texture_features = []
    sift_features = []
    brightness_features = []
    contrast_features = []
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index % sample_rate == 0:
            color_hist = extract_color_histogram(frame, bins=bins)
            lbp_hist = extract_lbp_features(frame, radii=radii, n_points=n_points)
            sift_desc = extract_sift_features(frame, n_clusters=sift_clusters)
            brightness, contrast = extract_brightness_contrast_features(frame)

            if sift_desc is not None:
                sift_features.append(sift_desc.flatten())
            color_features.append(color_hist)
            texture_features.append(lbp_hist)
            brightness_features.append(brightness)
            contrast_features.append(contrast)

        frame_index += 1

    cap.release()
    # Aggregate and average features
    color_features = np.mean(np.array(color_features), axis=0) if color_features else None
    texture_features = np.mean(np.array(texture_features), axis=0) if texture_features else None
    sift_features = np.mean(np.array(sift_features), axis=0) if sift_features else None
    average_brightness = np.mean(brightness_features) if brightness_features else None
    average_contrast = np.mean(contrast_features) if contrast_features else None

    return color_features, texture_features, sift_features, average_brightness, average_contrast


In [8]:
def process_all_videos(directory_path, sample_rate=30, bins=32, radii=[1, 3, 5], n_points=[8, 16, 24], sift_clusters=10):
    data_directory = os.path.join('/kaggle/working/', 'data')
    os.makedirs(data_directory, exist_ok=True)

    features_database = {}

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.mp4'):
            video_path = os.path.join(directory_path, file_name)
            color_features, texture_features, sift_features, average_brightness, average_contrast = process_video(video_path, sample_rate, bins, radii, n_points, sift_clusters)

            # Save the features
            save_features(color_features, os.path.join(data_directory, f'{file_name}_color_features.pkl'))
            save_features(texture_features, os.path.join(data_directory, f'{file_name}_texture_features.pkl'))
            save_features(sift_features, os.path.join(data_directory, f'{file_name}_sift_features.pkl'))
            save_features(average_brightness, os.path.join(data_directory, f'{file_name}_brightness.pkl'))
            save_features(average_contrast, os.path.join(data_directory, f'{file_name}_contrast.pkl'))

            # Store features in the database
            features_database[file_name] = (color_features, texture_features, sift_features, average_brightness, average_contrast)

            print(f"Processed and saved features for video {file_name}")

    return features_database

In [9]:
def save_features(features, file_name):
    """Save features to a file."""
    with open(file_name, 'wb') as f:
        pickle.dump(features, f)


In [10]:
def calculate_distance(color_features, texture_features, sift_features, brightness, contrast, other_color, other_texture, other_sift, other_brightness, other_contrast, color_weight=0.25, texture_weight=0.15, sift_weight=0.4, brightness_weight=0.1, contrast_weight=0.1):
    color_dist = np.linalg.norm(color_features - other_color)
    texture_dist = np.linalg.norm(texture_features - other_texture)
    sift_dist = np.linalg.norm(sift_features - other_sift)
    brightness_dist = abs(brightness - other_brightness)
    contrast_dist = abs(contrast - other_contrast)

    weighted_dist = (color_weight * color_dist + texture_weight * texture_dist + sift_weight * sift_dist + brightness_weight * brightness_dist + contrast_weight * contrast_dist)
    return weighted_dist

In [11]:
directory_path = '/kaggle/input/gogogomeomeomeo2/cho-meo'
process_all_videos(directory_path, sample_rate=30)

Processed and saved features for video 123.mp4
Processed and saved features for video 033.mp4
Processed and saved features for video 113.mp4
Processed and saved features for video 106.mp4
Processed and saved features for video 019.mp4
Processed and saved features for video 017.mp4
Processed and saved features for video 081.mp4
Processed and saved features for video 114.mp4
Processed and saved features for video 042.mp4
Processed and saved features for video 058.mp4
Processed and saved features for video 120.mp4
Processed and saved features for video 026.mp4
Processed and saved features for video 078.mp4
Processed and saved features for video 085.mp4
Processed and saved features for video 030.mp4
Processed and saved features for video 045.mp4
Processed and saved features for video 088.mp4
Processed and saved features for video 093.mp4
Processed and saved features for video 055.mp4
Processed and saved features for video 009.mp4
Processed and saved features for video 074.mp4
Processed and

{'123.mp4': (array([5.3289098e-01, 1.9714325e-03, 4.0943289e-04, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00], dtype=float32),
  array([0.00633797, 0.01885441, 0.01514532, 0.06499748, 0.11228543,
         0.12924558, 0.03492943, 0.02683561, 0.56420693, 0.02716186,
         0.01959132, 0.01485722, 0.01443348, 0.01271723, 0.01165711,
         0.01623789, 0.01939259, 0.02770378, 0.0418643 , 0.05907089,
         0.02092101, 0.02798837, 0.01756226, 0.01916286, 0.01868911,
         0.01498728, 0.50680518, 0.13635811, 0.02226935, 0.01251124,
         0.01100885, 0.00872343, 0.00685338, 0.00573149, 0.00581069,
         0.00660136, 0.00690266, 0.00828007, 0.01121267, 0.01589058,
         0.02306947, 0.03307427, 0.01214999, 0.01699865, 0.00943696,
         0.01507108, 0.00882249, 0.01616293, 0.00966051, 0.01152802,
         0.01434532, 0.0122539 , 0.47733004, 0.21830059]),
  array([19.403585, 23.23223 , 24.555882, ..., 16.545856, 17.175339,
         14.089089], dtype=float32),
  64